In [ ]:
import json
import os
import pathlib
import pandas as pd
from glob import glob
import nltk
from nltk import tokenize
import matplotlib.pyplot as plt
from scipy import stats
import math
import statistics
from bart_score import BARTScorer

def get_human_summary(summary_path):
    try:
        with open("../../../scripts/" + summary_path, encoding='utf-8') as f:
            summary_json = json.load(f)
            return summary_json["summary"]
    except Exception as e:
        print("Failed to read summary file: {}".format(e))
        return None

def calculate_F1():
    summaries_count = 0
    data = []
    used_files = []
    unique_books = set()
    unique_used_books = set()

    human_summaries = dict()
    #f = open(pathlib.Path("../../booksum/alignments/chapter-level-summary-alignments/chapter_summary_aligned_all_split.jsonl"),

    stopcount = 0
    
    f = open(pathlib.Path("../../../alignments/chapter-level-summary-alignments/chapter_summary_aligned_train_split.jsonl"),
             encoding='utf-8')

    for line in f:
        content = json.loads(line)
        if content['source'] == 'pinkmonkey':
            continue
        text = get_human_summary(content['summary_path'])
        if text is not None:
            try:
                human_summaries[content['summary_path']] = {
                    "chapter_title": content['book_id'],
                    "source": content['source'],
                    "summary_text": text,
                }
            except:
                continue

    
    print("Evaluating {} summary documents...".format(len(human_summaries)))
    print("Please send help!",type(human_summaries.items()))
    for summary_path, summary in human_summaries.items():


   

        # Get all related summary documents.
        unique_books.add(summary['chapter_title'])
        # Special case for Around the World in Eighty (80) Days
        if summary['chapter_title'] == "Around the World in Eighty Days":
            related_summaries = list(filter(
                lambda curr_summary: curr_summary['chapter_title'] == 'Around the World in 80 Days', human_summaries.values()))

        elif summary['chapter_title'] == "Around the World in 80 Days":
            related_summaries = list(filter(
                lambda curr_summary: curr_summary['chapter_title'] == 'Around the World in Eighty Days', human_summaries.values()))

        else:
            related_summaries = list(filter(lambda curr_summary: curr_summary['chapter_title'] == summary[
                                     'chapter_title'] and curr_summary['source'] != summary['source'], human_summaries.values()))
        # Remember which files have been used.
        used_files.extend(related_summaries)
        # print(summary['chapter_title'], summary['source'])
        # print(related_summaries)

        # if there are no related summary documents, then just print.
        if len(related_summaries) == 0:
            print("No related summary documents were found for {}.".format(
                summary['chapter_title']))
            continue

        # # Run the ROUGE command using the current summary as the reference and the related summaries as hypotheses.
        # # Print the scores and save them.
        related_summary_texts = [curr_summary['summary_text']
                                 for curr_summary in related_summaries]

        print("-------")

        ref_sents = tokenize.sent_tokenize(summary['summary_text'])

        human_summary_groups = []

        for curr in related_summary_texts:
            human_summary_groups.append(tokenize.sent_tokenize(curr))

        
        summary_scores = []

        #print("HSUM:", human_summary_groups)
        for i, summary in enumerate(human_summary_groups):
            for j, sent in enumerate(summary):
                if len(sent) == 1:
                    #print("human_summary_groups[i].get(sent)" ,human_summary_groups[i].get(sent))
                    human_summary_groups[i].remove(sent)
        #print("--------")
        print("HSUM Cleaned:", human_summary_groups)

        bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')

        for i, human_summary in enumerate(human_summary_groups):
            scentence_scores = []

            for ref_sent in ref_sents:
                best_score = -math.inf

                for hyp_sent in human_summary:
                    #print("ref: ", ref_sent)
                    #print("hyp: ", hyp_sent)
                    curr_score = bart_scorer.score([ref_sent], [hyp_sent]) #place inside array or else bart does it per word instead of scentence to scentence
                    #print(curr_score)
                    assert len(curr_score) == 1
                    if curr_score[0] > best_score:
                        best_score = curr_score[0]

                scentence_scores.append(best_score)

            mean_summary_score = statistics.mean(scentence_scores)
            summary_scores.append(mean_summary_score)

        print("Higest summary score: ", max(summary_scores))
        mean_sum_score = statistics.mean(summary_scores)
        print("Mean summary score: ", mean_sum_score)


        data.append([mean_sum_score, summary['chapter_title'], summary['source']])
        unique_used_books.add(summary['chapter_title'])
        summaries_count += 1

        stopcount += 1

        if stopcount >= 1:
            break


    return data, summaries_count, unique_books, unique_used_books

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

nltk.download('punkt')

data, summaries_count, unique_chapters, unique_used_chapters = calculate_F1()


In [ ]:
print("Unique chapters covered: {}".format(len(unique_chapters)))
print("Unique chapters used: {}".format(len(unique_used_chapters)))
BART_list = [data_item[0] for data_item in data]
BART_mean = sum(BART_list) / len(BART_list)
print("Mean BART: {}".format(BART_mean))
print()

# # Comment these out to avoid saving the csv files.
df = pd.DataFrame(data, columns=["BARTo", "chapter-title", "source"])
# Save file.
df.to_csv("../../csv_results/booksum_summaries/chapter-level-sum-comparison-results-test-bart.csv")

del()